# 02. Conventional RAG based on LangChain

Q: 논문의 내용을 전부 프롬프트에 넣어서 질문하면 대답을 잘할까?

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_teddynote import logging

logging.langsmith(project_name="Paper-Agent")

LangSmith 추적을 시작합니다.
[프로젝트명]
Paper-Agent


In [4]:
loader = PyMuPDFLoader(
    "../data/Retrieval Augmented Generation for Knowledge Intensive NLP Tasks.pdf"
)
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 19


In [5]:
paper_contents = ""

for i in range(len(docs)):
    paper_contents += docs[i].page_content

print(len(paper_contents))
print(paper_contents[:300])

69059
Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks
Patrick Lewis†‡, Ethan Perez⋆,
Aleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,
Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†
†Facebook AI Research; ‡University 


In [6]:
# 단계 6: 프롬프트 생성(Create Prompt)
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [7]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [8]:
chain = (
    {"context": lambda x: paper_contents, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
chain

{
  context: RunnableLambda(lambda x: paper_contents),
  question: RunnablePassthrough()
}
| PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nAnswer in Korean.\n\n#Context: \n{context}\n\n#Question:\n{question}\n\n#Answer:")
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10c411b90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10e4ed910>, root_client=<openai.OpenAI object at 0x10a7a78d0>, root_async_client=<openai.AsyncOpenAI object at 0x10c48cf90>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [10]:
# 체인 실행(Run Chain)
question = "이 논문의 제목 알려줘"
response = chain.invoke(question)
print(response)

논문의 제목은 "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks"입니다.


In [11]:
# 체인 실행(Run Chain)
question = "이 논문의 주제가 무엇이야?"
response = chain.invoke(question)
print(response)

이 논문은 지식 집약적인 자연어 처리(NLP) 작업을 위한 검색 증강 생성(Retrieval-Augmented Generation, RAG) 모델에 대해 다루고 있습니다. RAG 모델은 사전 학습된 시퀀스-투-시퀀스(seq2seq) 모델과 위키피디아의 밀집 벡터 인덱스를 결합하여, 검색 기반의 비매개변수 메모리를 활용하여 언어 생성을 수행합니다. 이 논문은 RAG 모델이 다양한 지식 집약적인 NLP 작업에서 최첨단 성능을 달성하며, 특히 개방형 도메인 질문 응답 작업에서 뛰어난 성과를 보인다고 설명합니다.


In [12]:
# 체인 실행(Run Chain)
question = "이 논문의 저자가 누구야?"
response = chain.invoke(question)
print(response)

이 논문의 저자는 Patrick Lewis, Ethan Perez, Aleksandra Piktus, Fabio Petroni, Vladimir Karpukhin, Naman Goyal, Heinrich Küttler, Mike Lewis, Wen-tau Yih, Tim Rocktäschel, Sebastian Riedel, Douwe Kiela입니다.


In [13]:
# 체인 실행(Run Chain)
question = "이 논문의 Abstract 설명해줘."
response = chain.invoke(question)
print(response)

이 논문의 초록에서는 대규모 사전 학습된 언어 모델이 매개변수에 사실적 지식을 저장할 수 있으며, 하위 NLP 작업에 맞게 미세 조정되었을 때 최첨단 결과를 달성할 수 있음을 보여줍니다. 그러나 이러한 모델의 지식 접근 및 조작 능력은 여전히 제한적이며, 지식 집약적인 작업에서는 작업별 아키텍처에 비해 성능이 뒤처집니다. 또한, 모델의 결정에 대한 출처를 제공하고 세계 지식을 업데이트하는 것은 여전히 연구 과제로 남아 있습니다. 이 논문에서는 사전 학습된 매개변수 및 비매개변수 메모리를 결합하여 언어 생성을 수행하는 RAG(Retrieval-Augmented Generation) 모델을 소개합니다. 이 모델은 사전 학습된 seq2seq 모델을 매개변수 메모리로, Wikipedia의 밀집 벡터 인덱스를 비매개변수 메모리로 사용합니다. 다양한 지식 집약적인 NLP 작업에서 모델을 미세 조정하고 평가하여 세 가지 공개 도메인 QA 작업에서 최첨단 성능을 달성했으며, 매개변수 seq2seq 모델과 작업별 검색 및 추출 아키텍처를 능가했습니다.


## Result

- 논문 전체 내용을 넣어버리면 원하는 결과가 나오기는 함
- 하지만 시간도 더 오래 걸리고 토큰 사용량도 많아짐(RAG 사용시 약 1,400 --> 전체 텍스트 사용시 약 18,000)